## support vector machine con grid search

In [ ]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.model_selection import LeaveOneOut
from sklearn import metrics
import pandas as pd
from matplotlib import pyplot as plt
from statistics import mean
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'


#setting up labels for dataset
labels = ['class', 'spec_num', 'eccentr', 'asp_ratio', 'elong', 'solidity', 'stoch_conv', 'iso_factor', 'max_ind_depth', 'lobedness', 'av_intensity', 'av_contr', 'smooth', 'third_mom', 'unif', 'entropy']
#importing data
df = pd.read_csv(r'./leaf/leaf.csv', header = None, names = labels)
# shuffling the dataframe
df = df.sample(frac=1).reset_index()
df = df.iloc[:, 1:17]   # needed to eliminate the old indexes column
#display(df)
#separating y from x
X = df.iloc[:, 2:16]
y = df.iloc[:, 0]

#display(X)

#### grid search con k-fold CV e LOOCV

domande:

- cos'è rbf (in kernel)?

- il rescaling delle variabili come si fa (probabilmente lo fa automaticamente)?

- C è la tolleranza (in particolare corrisponde alla prima o alla seconda formulazione di Medvet)?

- perché roc_auc_ovo e roc_auc_ovr non funzionano?

- cercare modo migliore per 'elencare' i regularization parameter

domande generali:

- ha senso tenere spec_number nel dataset?

- come si fa a mettere più di un effectivness index nella grid search?

In [ ]:
k = 5

# building the range of the regularization parameter (C)
exp = np.arange(-8, 8)
reg_param = 10.**exp

grid_param = {'C': reg_param,
              'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 
              'degree': np.arange(2, 5),
              'decision_function_shape': ['ovo', 'ovr']}

clf_cv = GridSearchCV(svm.SVC(), grid_param, cv=k, scoring='accuracy')
clf_cv.fit(X, y)
print(clf_cv.best_params_)
print(clf_cv.best_score_)

In [3]:
# building the range of the regularization parameter (C)
exp = np.arange(-8, 8)
reg_param = 10.**exp

grid_param = {'C': reg_param,
              'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 
              'degree': np.arange(2, 5),
              'decision_function_shape': ['ovo', 'ovr']}

clf_loocv = GridSearchCV(svm.SVC(), grid_param, cv=LeaveOneOut(), scoring='accuracy')
clf_loocv.fit(X, y)
print(clf_loocv.best_params_)
print(clf_loocv.best_score_)

KeyboardInterrupt: 